Script adapted from - https://github.com/stellaralgo/code-constantcontact/blob/main/docker/src/auth/newClientSetup.py
Part A = creates the authorization URL for the client
Part B = creates the token for the client. save this token in secrets manager - constant_contact_refresh_tokens_{TEAM_NAME}
For Part A, please change
  - PROFILE_NAME in line 11 to your AWS profile name
  - TEAM_NAME in line 12 to your team name
For Part B, please copy the code from the URL that client provides after authorizing the app and paste it in line 96

In [ ]:
import requests 
import random
import string
import json
import base64

PROFILE_NAME = 'default'
TEAM_NAME = "echl_kalamazoowings"
TEAM_NAME_SNAKE = TEAM_NAME.replace('-', '_')
stellar_client_id = "xxx"
stellar_client_secret = "xxx"

def getStellarCreds():
    """
    :return: the clientId and clientSecret
    """
    try:
        stellarclientid = stellar_client_id
        stellarclientsecret = stellar_client_secret
    except:
        raise Exception('Missing secret make sure it has been updated in AWS')
    return stellarclientid, stellarclientsecret


def getAuthLink(authUrl, scope, redirectUri):
    """
    :param authUrl: authorization URL from endpoints file
    :param scope: scope form endpoints file
    :param redirectUri: redirect Uri form endpoints file
    :return:
    """
    try:
        stellarclientid, _ = getStellarCreds()
    except:
        raise Exception('Missing clientid or secret for client, make sure it has been updated in AWS')
    state = ''.join(random.choices(string.ascii_letters + string.digits, k=16))
    auth_request_url = f'{authUrl}?client_id={stellarclientid}&scope={scope}' \
                       f'&response_type=code&state={state}&redirect_uri={redirectUri}'
    response = requests.get(auth_request_url)
    if response.status_code == 200:
        return response.url

def getToken(code, tokenUrl, redirectUri):
    """
    :param code:
    :return:
    """
    stellarclientid, stellarsecret = getStellarCreds()

    token_request_url = f'{tokenUrl}?code={code}&redirect_uri={redirectUri}&grant_type=authorization_code'
    token_credentials = base64.b64encode(f'{stellarclientid}:{stellarsecret}'.encode('ascii'))
    token_authheader = "Basic " + token_credentials.decode()
    token_headers = {
        'Accept': 'application/json',
        'Content-Type': 'application/x-www-form-urlencoded',
        'Authorization': f'{token_authheader}'
    }

    response = requests.post(token_request_url, headers=token_headers)
    if response.status_code == 200:
        access_token = json.loads(response.content.decode('utf-8'))

        print(f"Successfully retrieved token. Please save this token in secrets manager - constant_contact_refresh_tokens_{TEAM_NAME}")
        return access_token

    else:
        print('Bad Request')
        print(response.status_code)
        print(response.content)

f = open("./endpoints.json")
endpoints = json.load(f)
f.close()

authUrl = endpoints['auth']
tokenUrl = endpoints['token']
redirectUri = endpoints['redirectUri']
scope = endpoints['scope']

1. Run Part A first

In [ ]:
def partA():
    print(f'Auth URL for {TEAM_NAME}: {getAuthLink(authUrl, scope, redirectUri)}')

partA()

Click link - they log into Constant Contact with their own credentials - This will redirect them to a blank page which has a URL.
Have the client copy this URL and provide StellarAlgo.  This can be pasted directly into the meeting chat (this is not sensitive data)
This URL we are interested in the code which is used for 'code_from_client' in Part B below.

In [ ]:
code_from_client = "xxx"

def partB():
    test = getToken(code=code_from_client, tokenUrl=tokenUrl, redirectUri=redirectUri)
    print(f'Token for {TEAM_NAME}: {test}')

partB()

Use the code and update code_from_client
Run Part(B)
We are then interested in the REFRESH_TOKEN.
Save this token to 1Password and update in Secrets Manager.